# Serve the autoencoder and detect anomalous credit card activity

![overview-5.png](./images/overview-5.png)

## Query Model Repository for best mnist Model

In [1]:
from hops import model
from hops.model import Metric
MODEL_NAME="ccfraudmodel"
EVALUATION_METRIC="loss"

In [2]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MIN)

In [3]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ccfraudmodel
Model version: 1
{'loss': '1.74222993850708'}


## Create Model Serving of Exported Model

In [4]:
from hops import serving
from hops import hdfs

In [5]:
TOPIC_NAME = "credit_card_prediction_logs"

SERVING_NAME = MODEL_NAME
MODEL_PATH="/Models/" + best_model['name']
TRANSFORMER_PATH = "/Projects/" + hdfs.project_name() + "/Jupyter/card_activity_transformer.py"

response = serving.create_or_update(SERVING_NAME, MODEL_PATH, model_version=best_model['version'], artifact_version="CREATE",
                                    kfserving=True, transformer=TRANSFORMER_PATH, 
                                    topic_name=TOPIC_NAME, inference_logging="TRANSFORMED_AND_PREDICTIONS",
                                    instances=1, transformerInstances=1)

Inferring model server from artifact files: TENSORFLOW_SERVING
Creating serving ccfraudmodel for artifact /Projects/card_fraud_detection//Models/ccfraudmodel ...
Serving ccfraudmodel successfully created


In [6]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

ccfraudmodel


In [7]:
# Get serving status
serving.get_status(SERVING_NAME)

'Stopped'

# Start Model Serving Server

In [8]:
if serving.get_status(SERVING_NAME) == 'Stopped':
    serving.start(SERVING_NAME)

Starting serving with name: ccfraudmodel...
Serving with name: ccfraudmodel successfully started


In [21]:
import time
while serving.get_status(SERVING_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

## Sample credit card numbers

In [9]:
import hsfs

connection = hsfs.connection()
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
td_meta = fs.get_training_dataset("card_fraud_model", 1)

In [11]:
#`init_prepared_statement` method is needed to get serving_keys in case `get_serving_vector` has not beed called yet. This is not necessary for `get_serving_vector` method itself
td_meta.init_prepared_statement() 
td_meta.serving_keys

{'cc_num'}

For demo purposes lets prepare list of primary key values that we are interested in to buils feature vectore from online feature store

In [12]:
cc_nums = fs.get_feature_group("card_transactions", version=1).select("cc_num").read()

Lazily executing query: SELECT `fg0`.`cc_num`
FROM `card_fraud_detection_featurestore`.`card_transactions_1` `fg0`


In [13]:
cc_nums_inputs = cc_nums.sample(n=len(cc_nums)*2, replace=True)

In [14]:
len(cc_nums_inputs)

10800

#### Get serving vector and send to Prediction Requests to the Served Model using Hopsworks REST API

In [15]:
import numpy as np
TOPIC_NAME = serving.get_kafka_topic(SERVING_NAME)
print("Topic: " + TOPIC_NAME)

Topic: credit_card_prediction_logs


In [ ]:
i=0
for cc_num in cc_nums_inputs['fg0.cc_num']:
    data = { "signature_name": "serving_default", "instances": [{'cc_num': int(cc_num)}] }
    response = serving.make_inference_request(SERVING_NAME, data)
    if i % 2000 == 0:
        print(response)
    i+=1

{'predictions': [2.20478344]}
{'predictions': [0.955848038]}
{'predictions': [2.77835608]}
{'predictions': [1.0319711]}
{'predictions': [1.04669809]}
